In [1]:
import numpy as np
import pandas as pd

In [2]:
# Set Work Path
import os
path="C:/Users/zelic/Downloads/04.추가특허출원대상/fw"
os.chdir(path)

In [3]:
# Load Train Test Files
df_X_train = pd.read_pickle('df_X_train.pkl')
df_y_train = pd.read_pickle('df_y_train.pkl')
df_X_train['bias'] = 1

df_X_valid = pd.read_pickle('df_X_valid.pkl')
df_y_valid = pd.read_pickle('df_y_valid.pkl')
df_X_valid['bias'] = 1

In [4]:
# Load Model

from keras.models import model_from_json

# load json and create model
json_file = open('model_architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("best_model_dropout30.h5")

print("Model Loading Complete")

Model Loading Complete


In [5]:
result_train = pd.DataFrame(df_y_train, columns = {'delayed'})
result_train['normal'] = 1 - result_train['delayed']
result_train['normal_prob'] = 1 - loaded_model.predict(df_X_train)
result_train['delayed_prob'] = 1 - result_train['normal_prob']

result_train['predict'] = [1 if x >= 0.5 else 0 for x in result_train['delayed_prob']]

result_train['normal_prob'] = [0.99999 if x >= 0.99999 else x for x in result_train['normal_prob']]
result_train['normal_prob'] = [0.00001 if x <= 0.00001 else x for x in result_train['normal_prob']]
result_train['delayed_prob'] = [0.99999 if x >= 0.99999 else x for x in result_train['delayed_prob']]
result_train['delayed_prob'] = [0.00001 if x <= 0.00001 else x for x in result_train['delayed_prob']]

result_train['odds'] = result_train['normal_prob'] / result_train['delayed_prob']
#result_train.sort_values(by = 'delayed_prob', ascending = False, inplace = True)
result_train = result_train.reset_index(drop = True)
pdo = 50
result_train['score_temp'] = pdo / np.log(2) * np.log(result_train['odds'])
result_train_min_score = np.min(result_train['score_temp'])
result_train['score'] = result_train['score_temp'] - result_train_min_score

18859/18859 [==============================] - 65s 3ms/step


In [6]:
x = result_train['score'].values.reshape(-1, 1)
y = result_train['normal']

In [7]:
from sklearn import tree
grading_dt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=None,max_leaf_nodes=10)
grading_dt.fit(x, y)

DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=10)

In [8]:
# save the model to disk
import pickle

filename = 'grading_by_risk_score.sav'
pickle.dump(grading_dt, open(filename, 'wb'))

In [9]:
# load the model from disk
filename = 'grading_by_risk_score.sav'
loaded_grading_dt = pickle.load(open(filename, 'rb'))

In [10]:
result_train['grade_temp'] = loaded_grading_dt.apply(x)

In [11]:
grade_result = result_train[['score','grade_temp']].groupby(['grade_temp']).min()

In [12]:
grade_result = grade_result.sort_values(by = 'score', ascending = False).reset_index().reset_index()

In [13]:
grade_result = grade_result.rename(columns={'index': 'grade_final'}) 
grade_result['grade_final'] = grade_result['grade_final'] + 1
grade_result = grade_result[['grade_temp', 'grade_final']]
grade_result.to_pickle('grade_result.pkl')

In [14]:
grade_result

,grade_temp,grade_final
0,14,1
1,18,2
2,17,3
3,8,4
4,7,5
5,12,6
6,11,7
7,16,8
8,15,9
9,9,10


In [15]:
result_train = pd.merge(result_train, grade_result, on = 'grade_temp')

In [16]:
result_train[['delayed','grade_final']].groupby(['grade_final']).agg(['sum','count'])

delayed        
                sum   count
grade_final                
1               983  128694
2              1445   85189
3              1388   52402
4              4567  108007
5              7584   98977
6              7091   55930
7              8301   41612
8              3571   12528
9              4539   12480
10             4002    7654

In [17]:
result_valid = pd.DataFrame(df_y_valid, columns = {'delayed'})
result_valid['normal'] = 1 - result_valid['delayed']
result_valid['normal_prob'] = 1 - loaded_model.predict(df_X_valid)
result_valid['delayed_prob'] = 1 - result_valid['normal_prob']

result_valid['predict'] = [1 if x >= 0.5 else 0 for x in result_valid['delayed_prob']]

result_valid['normal_prob'] = [0.99999 if x >= 0.99999 else x for x in result_valid['normal_prob']]
result_valid['normal_prob'] = [0.00001 if x <= 0.00001 else x for x in result_valid['normal_prob']]
result_valid['delayed_prob'] = [0.99999 if x >= 0.99999 else x for x in result_valid['delayed_prob']]
result_valid['delayed_prob'] = [0.00001 if x <= 0.00001 else x for x in result_valid['delayed_prob']]

result_valid['odds'] = result_valid['normal_prob'] / result_valid['delayed_prob']
#result_valid.sort_values(by = 'delayed_prob', ascending = False, inplace = True)
result_valid = result_valid.reset_index(drop = True)
pdo = 50
result_valid['score_temp'] = pdo / np.log(2) * np.log(result_valid['odds'])
result_valid_min_score = np.min(result_valid['score_temp'])
result_valid['score'] = result_valid['score_temp'] - result_valid_min_score

4715/4715 [==============================] - 20s 4ms/step


In [18]:
result_valid['grade_temp'] = loaded_grading_dt.apply(result_valid['score'].values.reshape(-1, 1))
result_valid = pd.merge(result_valid, grade_result, on = 'grade_temp')

In [19]:
result_valid[['delayed','grade_final']].groupby(['grade_final']).agg(['sum','count'])

delayed       
                sum  count
grade_final               
1               398  31949
2               464  21485
3               412  13042
4              1348  26874
5              1910  24810
6              1773  14130
7              1896  10428
8               825   3270
9               974   3044
10              879   1837